# GeoNet news story on eruption at HT-HH

In [ ]:
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client
import obspy
from obspy.signal.detrend import polynomial

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime as dt

In [ ]:
client = Client('http://service-nrt.geonet.org.nz')

In [ ]:
t = UTCDateTime("2022-01-15T03:45:00")
tint = 60 * 60 * 12

### Air pressure COVZ

In [ ]:
sta = client.get_waveforms("NZ", "COVZ", "30", "HDF", t, t + tint, attach_response=True)
tra = sta[0]

In [ ]:
tra.stats.starttime+=13*60*60 #add 13 hours to convert to NZDT

In [ ]:
gaina = tra.stats.response.instrument_sensitivity.value

In [ ]:
tra.data = 600 + tra.data/gaina #divide out gain and add 600 hPa offset

In [ ]:
meantra = tra.data.mean()
tra.data -= meantra

In [ ]:
print (tra.data.min(), tra.data.max()) #units hPa
print ('peak-to-trough')
print (tra.data.max() - tra.data.min())

In [ ]:
tra.plot()

### Seismic

In [ ]:
sts = client.get_waveforms("NZ", "COVZ", "10", "HHZ", t, t + tint, attach_response=True)
trs = sts[0]

In [ ]:
trs.stats.starttime+=13*60*60 #add 13 hours to convert to NZDT

In [ ]:
gains = trs.stats.response.instrument_sensitivity.value

In [ ]:
trs.data = (trs.data / gains) * 1e9 #nm/s
# trs.remove_response()

In [ ]:
meantrs = trs.data.mean()
trs.data -= meantrs

In [ ]:
trs.plot()

In [ ]:
trsfilt = trs.copy()

In [ ]:
trsfilt.filter('lowpass', freq=0.05, corners=2)

In [ ]:
trsfilt.plot()

### Coastal sea level GIST

In [ ]:
stc = client.get_waveforms("NZ", "GIST", "40", "LTZ", t, t + tint, attach_response=True)
trc = stc[0]

In [ ]:
trc.stats.starttime+=13*60*60 #add 13 hours to convert to NZDT

In [ ]:
gainc = trc.stats.response.instrument_sensitivity.value
gainc

In [ ]:
trc.data = trc.data/gainc

In [ ]:
polynomial(trc.data, order=3) #modified in place

In [ ]:
trc.plot()

In [ ]:
# trc_filt = trc.copy()
# trc_filt.detrend(type='linear')
# # trc_filt.filter('lowpass', freq=0.01, corners=2, zerophase=True)
# trc_filt.plot()

### Visualize THIS IS IN NZDT

In [ ]:
fig,(axs, axa,axc) = plt.subplots(nrows=3, ncols=1, figsize=(20,12))
plt.subplots_adjust(hspace=0.1)

#seismic wave ).05 Hz LP filtered version so cleaner signal
axs.plot(trsfilt.times("matplotlib"), trsfilt.data, color='slategrey')
axs.xaxis_date()
axs.set_xticks([])
axs.set_ylim(bottom=trsfilt.data.min(), top=trsfilt.data.max())

axs.text(dt.datetime(2022,1,15,19,10), -5000, 'first arrival after about 4 minutes', fontsize=14, color='slategrey', va='bottom', ha='left')


axs.axvline(dt.datetime(2022,1,15,17,8),linestyle='--', color='firebrick')
axs.text(dt.datetime(2022,1,15,17,8), 2000, '5:08 PM', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)
axs.text(dt.datetime(2022,1,15,17,8), -7500, 'eruption start', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)

#volcano image, can't work in data coordinates as having issue with datetime, fudge exis coordinates so it look okay
image = plt.imread('volcano.png', format='png')
axin = axs.inset_axes([-0.022,1.02, .2, .2])
axin.imshow(image)
axin.axis('off')

axs.text(dt.datetime(2022,1,16,4,45), 6000, 'Earthquake ground wave \n central North Island (lowpass filter 20 s)', color='slategrey', ha='right',  va='top', fontsize=20)
axs.set_ylabel('Ground velocity (nano-m/s)')

#air pressure wave
axa.plot(tra.times("matplotlib"), tra.data, color='cornflowerblue')
axa.xaxis_date()
axa.set_xticks([])
axa.set_ylim(bottom=tra.data.min(), top=tra.data.max())

axa.axvline(dt.datetime(2022,1,15,17,8),linestyle='--', color='firebrick')
# axa.text(dt.datetime(2022,1,15,17,8), 3.7, 'E', color='firebrick', ha='center', fontsize=20)

axa.axvline(dt.datetime(2022,1,15,19,0),linestyle='--', color='grey')
axa.text(dt.datetime(2022,1,15,19,0), -2, '7 PM', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)
axa.text(dt.datetime(2022,1,15,19,0), 0.2, 'first arrival', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)

axa.text(dt.datetime(2022,1,16,4,45), 2.5, 'Air pressure wave in atmosphere\n central North Island', color='cornflowerblue', ha='right',  va='top', fontsize=20)

axa.set_ylabel('Air pressure change (hPa)')

#tsunami wave
axc.plot(trc.times("matplotlib"), trc.data, color='blue')
axc.xaxis_date()
axc.axvline(dt.datetime(2022,1,15,17,10),linestyle='--', color='firebrick')

axc.axvline(dt.datetime(2022,1,15,20,0),linestyle='--', color='grey')
axc.text(dt.datetime(2022,1,15,20,0), -0.7, '8 PM', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)
axc.text(dt.datetime(2022,1,15,20,0), 0.4, 'first\n arrival', color='dimgray', va='bottom', ha='right', fontsize=14, rotation=90)

axc.set_ylabel('Wave height (m)')
axc.set_yticks([-1, -0.5, 0, 0.5, 1])

axc.text(dt.datetime(2022,1,15,21,0), 0.95, 'Tsunami wave in ocean\nat Gisborne', color='blue', ha='left',  va='top', fontsize=20)

plt.suptitle('GeoNet Recording the Tonga Eruption in Aotearoa/New Zealand, 2022 January 15', y=0.95, fontsize=20)
axs.set_title('Data source: https://www.geonet.org.nz/data/tools/FDSN', y=1.05, fontsize=16, color='grey')

plt.savefig('tonga_impact.png', facecolor='white', dpi=300, bbox_inches='tight')
